In [ ]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# third party imports
import pandas as pd 
# python imports
from datetime import date
# local imports
from packages.rnn import RNN
from packages.dataset import DatasetConstructor


# Initiate Device
device = device = torch.device('cpu')

In [ ]:
SEQUENCE_LENGTH = 10


# GET S&P 500 Stocks for training
STOCK_LIST = pd.read_excel('stocks.xlsx')['Stock'].tolist()

dataset = DatasetConstructor(
    STOCK_LIST, date(2000, 1, 1), date(2021, 1, 1),SEQUENCE_LENGTH, 100)

training_batches = dataset.dataset_to_training_tensor()

In [ ]:
num_layers = 1
hidden_size = 2

model = RNN(dataset.input_size, hidden_size, num_layers, dataset.input_size)

EPOCHS = 12
optimizer = optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

last_x = None
for epoch in range(EPOCHS):
    for data in training_batches:

        X_tensor = data['tensor']
        labels = data['labels']
        labels = labels.long()

        output = model(X_tensor)

        loss = criterion(output, labels)
        model.zero_grad()  
        loss.backward()
        optimizer.step()

        last_x = X_tensor

    print(loss)

torch.save(model.state_dict(),'./epochs/model_'+str(SEQUENCE_LENGTH)+ \
    's_' + str(num_layers) + 'l_' +str(hidden_size) + 'h.pt')
    
all = DatasetConstructor(
    ['ADBE','MSFT','PYPL'], date(2021, 1, 1), date.today(),15, 1)
adbe = DatasetConstructor(
    ['ADBE'], date(2021, 1, 1), date.today(),15, 1)
msft = DatasetConstructor(
    ['MSFT'], date(2021, 1, 1), date.today(),15, 1)
pypl = DatasetConstructor(
    ['PYPL'], date(2021, 1, 1), date.today(),15, 1)

testing_datasets = {
    'ALL' : all,
    'ADBE': adbe,
    'MSFT': msft,
    'PYPL': pypl
}


In [ ]:


for dset in testing_datasets:


    testing_batches = testing_datasets[dset].dataset_to_training_tensor()


    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for data in testing_batches:
            X_tensor = data['tensor']
            labels = data['labels']
            outputs = model(X_tensor)

            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'{dset} Accuracy: {acc} %')